In [1]:
import sys
import os
sys.path.append('../models')

In [2]:
!pip install -q tensorflow-model-optimization

In [3]:
from models.resnet20 import build_resnet20

In [4]:
from traning.train_resnet20 import train_model
train_model()  # if you defined it that way

ModuleNotFoundError: No module named 'resnet20'